In [1]:
import seqio
from seqio import Mapping
import tensorflow.compat.v1 as tf
from pathlib import Path
from datasets import Dataset, load_dataset
import datasets
from transformers import AutoTokenizer
ROOT = Path.cwd().parents[0]
def feature_to_spec(feature, length=False):
    if isinstance(feature, datasets.ClassLabel):
        return tf.TensorSpec(shape=() if not length else (None if length == -1 else length,), dtype=tf.int64)
    elif isinstance(feature, datasets.Value):
        print(feature)
        return tf.TensorSpec(
            shape=() if not length else (None if length == -1 else length,), dtype=getattr(tf.dtypes, feature.dtype)
        )
    elif hasattr(feature, "dtype") and hasattr(feature, "shape"):
        return tf.TensorSpec(shape=feature.shape, dtype=feature.dtype)
    elif isinstance(feature, datasets.Sequence):
        return feature_to_spec(feature.feature, length=feature.length)
    elif isinstance(feature, list):
        return [feature_to_spec(f, length=length) for f in feature]
    elif isinstance(feature, dict):
        return {k: feature_to_spec(v, length=length) for k, v in feature.items()}
    else:
        raise ValueError(f"Unparseable feature type {type(feature)}")
        
def hf_dataset_to_tf_dataset(dataset):
    return tf.data.Dataset.from_generator(
        dataset.__iter__, output_signature={k: feature_to_spec(v) for k, v in dataset.features.items()}
    )
data_path = ROOT.joinpath('data','MBPP')
dataset = load_dataset('json',data_files={'train':str(data_path.joinpath('train.jsonl')),"validation":str(data_path.joinpath('validation.jsonl'))})
def get_tf_dataset(split, shuffle_files):
    # HF datasets does not support file-level shuffling
    del shuffle_files
    
    dataset = load_dataset('json',data_files={'train':str(data_path.joinpath('train.jsonl')),"validation":str(data_path.joinpath('validation.jsonl'))})
    dataset = dataset.map(lambda x,idx: {'idx':idx,**{k:v for k,v in x.items() if k != 'challenge_test_list'}},with_indices=True, remove_columns=dataset['train'].column_names)
    return hf_dataset_to_tf_dataset(dataset[split])

@seqio.map_over_dataset
def fix_col(
    sample: Mapping[str, tf.Tensor]
) -> Mapping[str, tf.Tensor]:
    return {
      'inputs': sample['text'],
      'targets': sample['code'],
  }

2022-01-09 15:56:40.899945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64
2022-01-09 15:56:40.899958: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Using custom data configuration default-ef56cc1ea23e9418
Reusing dataset json (/home/gabe/.cache/huggingface/datasets/json/default-ef56cc1ea23e9418/0.0.0/c90812beea906fcffe0d5e3bb9eba909a80a998b5f88e9f8acbd320aa91acfde)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
tokenizer= AutoTokenizer.from_pretrained('t5-base')
vocab = list(tokenizer.vocab)

split_mapping = {"train":"train","validation":"validation"}

In [3]:
data_source = seqio.FunctionDataSource(
        get_tf_dataset,
        splits=list(split_mapping.keys()),
        num_input_examples={s: len(dataset[split_mapping[s]]) for s in split_mapping.keys()},
    )
import os
seqio_vocab = seqio.SentencePieceVocabulary(tokenizer.vocab_file,extra_ids=100)

output_features = {
    "inputs": seqio.Feature(seqio_vocab, add_eos=False, dtype=tf.int32),
    "targets": seqio.Feature(seqio_vocab, add_eos=True, dtype=tf.int32),
}
preprocessors = [
    fix_col,
    seqio.preprocessors.tokenize,
    seqio.preprocessors.append_eos,
    seqio.CacheDatasetPlaceholder(required=False),
]

import sklearn.metrics
def acc(targets,predictions):
    return {"accuracy": 100*sklearn.metrics.accuracy_score(targets, predictions)}
seqio.TaskRegistry.remove('testing')
seqio.TaskRegistry.add(
    "testing",
    data_source,
    preprocessors=preprocessors,
    output_features=output_features,
    metric_fns=[acc],
    postprocess_fn=lambda f: f,
)

In [20]:
tf_dataset: tf.data.Dataset = seqio.get_dataset(
    mixture_or_task_name="testing",
    task_feature_lengths={"targets": 512,"inputs":512},
    dataset_split="train",
    shuffle=False,
    feature_converter=seqio.EncDecFeatureConverter()
)

  0%|          | 0/2 [00:00<?, ?it/s]

INFO:absl:Automatically caching small dataset in memory: 'testing:train'


Value(dtype='string', id=None)
Value(dtype='string', id=None)
Value(dtype='int64', id=None)
Value(dtype='string', id=None)
Value(dtype='string', id=None)
Value(dtype='int64', id=None)


INFO:absl:The output dataset from seqio.get_dataset has the following features
INFO:absl:feature: encoder_input_tokens 	 shape: [512] 	 dtype: int32
INFO:absl:feature: decoder_target_tokens 	 shape: [512] 	 dtype: int32
INFO:absl:feature: decoder_input_tokens 	 shape: [512] 	 dtype: int32
INFO:absl:feature: decoder_loss_weights 	 shape: [512] 	 dtype: int32
INFO:absl:feature: encoder_segment_ids 	 shape: [512] 	 dtype: int32
INFO:absl:feature: decoder_segment_ids 	 shape: [512] 	 dtype: int32
INFO:absl:feature: encoder_positions 	 shape: [512] 	 dtype: int32
INFO:absl:feature: decoder_positions 	 shape: [512] 	 dtype: int32


In [14]:
import torch
from transformers import AutoModelForSeq2SeqLM
model = 

SyntaxError: invalid syntax (3881999250.py, line 3)

In [21]:
[x for _,x in zip(range(1),tf_dataset)]

[{'encoder_input_tokens': <tf.Tensor: shape=(512,), dtype=int32, numpy=
  array([ 8733,     3,     9,     3,   102,    63,   189,   106,  1681,
            12,   253,  2559,  4505,    13,  2580,    13,     3,     9,
           787,   381,     5,  8733,     3,     9,  1681,    12,  3476,
             8,  3282,  7321,    16,     8,  4838,     3,  1496,  1054,
             3,    17,   413,   109,     5,  8733,     3,     9,  1681,
            12,  5755,     3,    17,   413,   109,   139,   570,    57,
          2651,     8,   787,  6108,   227,   334,  3282,     5,  8733,
             3,     9,  1681,    12,   253,     8,  4505,    51,   257,
            13,     3,    17,   413,   109,  2479,    16,     8,   787,
             3,    17,   413,   109,   570,     5,  8733,     3,     9,
          1681,    12,   691,     3,    99,   132,    19,     3,     9,
           769,  2244,    28,  4505,  1227, 23448,    57,     3,    51,
             5,     0,     0,     0,     0,     0,     0,     0,